In [1]:
import os
os.chdir("..")

In [110]:

import json
import numpy as np
import math
from typing import Dict, List, Tuple
import copy
from itertools import combinations
from collections import Counter

import logging
from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder
from ConflictDetector import ConflictDetector, ConflictPredicator


import plotly.graph_objects as go
from plotly.subplots import make_subplots

test_projects = [
    # "House1",
    "House3",
    "House4",
    "House8",
    "House9",
    "House15",
    # "House18",
    # "House20",
]
ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "weatherDesc#CAT": {},
            # WEEKDAY_CTX: {
            #     "range": (0, 6),
            #     "interval": 1,
            # },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # }
        })
capacity = {
    "TV":1,
    "WashingMachine":1,
    "PC": 1,
}
BOOL_SIM = False
BOOL_UMASS= False
ccp_alpha = 2e-4
test_dates = generate_test_date(test_projects, test_ratio = 0.4, true_random=False, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
# test_dates = []


In [111]:
habit_groups = {}
for p in test_projects:
    habit_groups[p], grid_data = test_umass(test_project=p, ctx_info=ctx_info, ccp_alpha=ccp_alpha, test_dates=test_dates, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)

In [112]:
#Exps:
#   1. Contrived environment with injected conflicts:
#   2. Real dataset with groundtruth conflicts:
#   3. observations about the algorithm: with different threshold, different capacity for the devices, different number of users.

In [113]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))
        # if d != "TV":
        #     continue
        # for g in d_groups:
        #     print(ctx_info.coor_box_to_range(g["box"][0] + g["box"][1]))
        #     print(g["dis"])


The habit groups found in House3
WashingMachine 5
TV 12
The habit groups found in House4
PC 3
WashingMachine 1
TV 13
The habit groups found in House8
PC 2
WashingMachine 3
TV 6
The habit groups found in House9
WashingMachine 2
TV 10
The habit groups found in House15
TV 10
WashingMachine 1


In [114]:
c_detector = ConflictDetector(ctx_info, capacity)
final_conflicts = c_detector.predict_conflict_scenarios(habit_groups)
print({x:len(final_conflicts[x]) for x in final_conflicts})

{'TV': 171, 'WashingMachine': 37, 'PC': 4}


In [115]:
import plotly.express as px
# for x in final_conflicts["Window"]:
#     print((ctx_info.coor_box_to_range(x["box"]),x["prob"]))
probs_i = [(x["prob"], i) for i, x in enumerate(final_conflicts["TV"])]
# print(sorted(probs_i))
# fig = px.histogram(probs)
# fig.show()

In [116]:
import copy
# Make ground truth:

gt_ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            "weatherDesc#CAT": {},
            # WEEKDAY_CTX: {
            #     "range": (0, 6),
            #     "interval": 1,
            # },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # },
        })
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(p, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
    device_events[p] = device_evts

gtconflict_cfg = {
    "context_info": gt_ctx_info,
    "capacity": capacity
}
# test_dates = {}

conflict_finder = GtConflictFinder(gtconflict_cfg)
gt_conflicts, test_state_cnt = conflict_finder.get_Gt_conflict(ctx_evts, device_events, test_dates)
print(len(gt_conflicts))

6309


In [117]:

# TEST_RATIO = 0.3
# conflict_time = [x['cur_time'] for x in conflicts]
# end_time = max(conflict_time)
# start_time = min(conflict_time)
# print(start_time)
# print(end_time)
# total_time_range = end_time - start_time
# test_start = end_time - total_time_range * TEST_RATIO
# print(test_start)

conflict_device = {
    d:[]
    for d in capacity
}

conflict_state_device = {
    d:{}
    for d in capacity
}
MIN_TEST_OBS = 20
cnts = []
for c in gt_conflicts:
    d = c["device"]
    conflict_device[d].append(c)
    s = gt_ctx_info.get_coor_by_ctx(c["ctx"])
    users = frozenset(c["device_states"].keys())
    cnts.append(test_state_cnt[d][s])
    if test_state_cnt[d][s] < MIN_TEST_OBS:
        continue
    if s not in conflict_state_device[d]:
        conflict_state_device[d][s] = {}
    conflict_state_device[d][s][users] = conflict_state_device[d][s].get(users, 0) + 1

    # if conflict_state_device[d][s][users] > test_state_cnt[d][s]:
    #     print(conflict_state_device[d][s][users], test_state_cnt[d][s])
    #     print(users, d, c["cur_time"], s)
    
print({d:len(c) for d,c in conflict_device.items()})
print({d:len(c) for d,c in conflict_state_device.items()})
print(len(test_dates))
print(Counter(cnts))

{'TV': 6223, 'WashingMachine': 85, 'PC': 1}
{'TV': 56, 'WashingMachine': 30, 'PC': 1}
203
Counter({123.0: 1526, 101.0: 1381, 66.0: 767, 58.0: 625, 27.0: 319, 106.0: 267, 136.0: 206, 14.0: 205, 47.0: 135, 7.0: 130, 53.0: 124, 104.0: 117, 46.0: 111, 6.0: 84, 103.0: 76, 113.0: 44, 37.0: 38, 1.0: 31, 39.0: 26, 54.0: 22, 36.0: 22, 10.0: 15, 41.0: 13, 3.0: 10, 8.0: 7, 2.0: 5, 29.0: 1, 88.0: 1, 11.0: 1})


In [118]:
all_users = test_projects
all_devices = capacity.keys()
user_pairs = list(combinations(all_users, 2))
conflict_predicator = ConflictPredicator(ctx_info, final_conflicts)
exp_cnt = 0
exp_gt_c = 0
exp_result = {d:[0,0,0,0,0] for d in all_devices}
max_p = 0
all_state_cnt = 0
for d in all_devices:
    it = np.nditer(test_state_cnt[d], flags=['multi_index'])
    for count in it:
        all_state_cnt += count
        if count < MIN_TEST_OBS:
            continue
        state = it.multi_index
        for u_pair in user_pairs:
            gt_prob = 0.
            if (d in conflict_state_device) and \
                (state in conflict_state_device[d]) and \
                (frozenset(u_pair) in conflict_state_device[d][state]):

                gt_prob =  float(conflict_state_device[d][state][frozenset(u_pair)]) / count
            gt_ctx_snapshot = gt_ctx_info.coor_to_snapshot(state)

            pred_prob = conflict_predicator.get_prob_conflict(ctx_info.get_coor_by_ctx(gt_ctx_snapshot), u_pair, d)
            exp_cnt += 1
            if gt_prob > 0:
                # if gt_prob > 0.5:
                #     print(gt_ctx_info.coor_to_snapshot(state).values(), gt_prob, pred_prob, u_pair)
                exp_gt_c += 1
                exp_result[d][0] += abs(pred_prob - gt_prob)
                exp_result[d][1] += 1
                exp_result[d][4] += gt_prob
                if gt_prob > 1:
                    print(gt_prob, pred_prob, d, u_pair)
                max_p = max(gt_prob, max_p)

            else:
                exp_result[d][2] += pred_prob - gt_prob
                exp_result[d][3] += 1

for d in exp_result:
    exp_result[d].append((exp_result[d][0] + exp_result[d][2]) / (exp_result[d][1] + exp_result[d][3]))
    if exp_result[d][1] > 0:
        exp_result[d][0] = exp_result[d][0] / exp_result[d][1]
        exp_result[d][4] = exp_result[d][4] / exp_result[d][1]
    if exp_result[d][3] > 0:
        exp_result[d][2] = exp_result[d][2] / exp_result[d][3]
    
print(max_p)
print(exp_result)
print(all_state_cnt)

0.7037037037037037
{'TV': [0.0455070010026292, 403, 0.002236702556416019, 287, 0.18902536598890066, 0.02750906527210285], 'WashingMachine': [0.015810878650117165, 58, 0.0007884408026943219, 632, 0.018806538920318767, 0.0020511964478400102], 'PC': [0.007314298931541169, 1, 3.0476421730008037e-05, 689, 0.009615384615384616, 4.103268623698073e-05]}
14582.0


Run grid search for the optimal ccp_alpha

In [ ]:
from refit_alpha_test import *

ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "weatherDesc#CAT": {},
            # WEEKDAY_CTX: {
            #     "range": (0, 6),
            #     "interval": 1,
            # },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # },
        })
gt_ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            "weatherDesc#CAT": {},
            # WEEKDAY_CTX: {
            #     "range": (0, 6),
            #     "interval": 1,
            # },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # },
        })


def alpha_nonLinear_generator(alpha_min, alpha_max, alpha_step):
    auc_values = []
    alpha = alpha_min
    while alpha <= alpha_max:
        yield alpha
        if alpha_step <= alpha / 10:
            alpha_step *= 10
        alpha += alpha_step

print("Run test accuracy for context " + str(ctx_info.get_all_ctx_ordered()))
alpha_generator = alpha_nonLinear_generator(1e-7, 0.001, 1e-7)
acc_values = test_acc_alpha(ctx_info, gt_ctx_info, alpha_generator)


Run test accuracy for context ['min_of_day#NUM']


TypeError: '<' not supported between instances of 'str' and 'datetime.datetime'